In [2]:
import os 
import scipy.io
import numpy as np
import pandas as pd
from sklearn import model_selection, svm, naive_bayes
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
import serial
import time
import pyautogui

directory = './murillo'
filenames = ['hang_loose', 'pinch', 'punch', 'rest', 'thumbs_up', 'victory']

np.random.seed(0)

C:\Users\guilh\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [10]:
threshold_multiplication = 1950*2100
threshold_minus = 2100-1900
def sgn(x1, x2):
    if(x1*x2 >= threshold_multiplication and abs(x1-x2) >= threshold_minus):
        return 1
    else:
        return 0

dataframe = []

def feature_extraction(xiao_data):
    rms = np.sqrt(np.mean(xiao_data**2))
    mav = np.sum(abs(xiao_data))/len(xiao_data)
    wl = np.sum([xiao_data[n] - xiao_data[n-1] for n in range(1, len(xiao_data)-1)])
    intensity = np.log(rms/len(xiao_data))
    zc = np.sum([sgn(xiao_data[n], xiao_data[n+1]) for n in range(1, len(xiao_data)-1)])
    var = np.sum(xiao_data**2)/(len(xiao_data)-1)
    ssc = np.sum([xiao_data[n] - xiao_data[n-1] if (xiao_data[n] - xiao_data[n-1])>=0 else 0  for n in range(1, len(xiao_data) - 1)])  
    return [rms, mav, wl, intensity, zc, var, ssc]

for file in filenames:
    with open(os.path.join(directory, f'{file}_continuous.txt'), "r") as f:
        lines = f.readlines()
    # extract the values after "->" and convert them to integers
    values = [int(float(line.split("->")[1].strip())) for line in lines]
    for i in range(0, len(values)-200, 10):
        x = values[0+i:200 + i]
        x = np.array(x, dtype=np.float64)

        feature_data = feature_extraction(x)
        feature_data.append(file)
        dataframe.append(feature_data)
            
df = pd.DataFrame(dataframe, columns=["RMS", "MAV", "WL", "INTENSITY", "ZC", "VAR", "SSC", "LABEL"])
df.head()

,RMS,MAV,WL,INTENSITY,ZC,VAR,SSC,LABEL
0,2480.862518,2478.625,113.0,2.518044,10,6.185607e+06,7600.0,hang_loose
1,2482.960650,2480.730,93.0,2.518890,10,6.196074e+06,7603.0,hang_loose
2,2482.638839,2480.455,34.0,2.518760,11,6.194468e+06,7551.0,hang_loose
3,2480.937746,2478.730,423.0,2.518075,10,6.185982e+06,7481.0,hang_loose
4,2483.677632,2481.860,-170.0,2.519178,7,6.199653e+06,6884.0,hang_loose


In [11]:
X = df.drop('LABEL', axis=1)
y = df['LABEL']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [ ]:
SVM = svm.SVC(kernel='linear',gamma='auto')
SVM.fit(X_train, y_train) 
predictions_SVM = SVM.predict(X_test)
print("SVM Accuracy Score -> {accuracy:.1f}%".format(accuracy = accuracy_score(predictions_SVM, y_test)*100))

In [ ]:
from sklearn.preprocessing import MinMaxScaler #fixed import
Naive = naive_bayes.MultinomialNB()
scaler = MinMaxScaler()
Naive.fit(scaler.fit_transform(X_train.values),y_train)# predict the labels on validation dataset
predictions_NB = Naive.predict(scaler.transform(X_test))# Use accuracy_score function to get the accuracy
print("NB Accuracy Score -> {accuracy:.1f}%".format(accuracy = accuracy_score(predictions_NB, y_test)*100))

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()
clf.fit(X_train, y_train)

predictions_NN = clf.predict(X_test)
print("Neural network Accuracy Score -> {accuracy:.1f}%".format(accuracy = accuracy_score(predictions_NN, y_test)*100))

In [21]:
model_to_be_save = SVM
f = open('classifier.pickle', 'wb')
pickle.dump(model_to_be_save, f)
f.close()

In [8]:
f = open('classifier.pickle', 'rb')
classifier = pickle.load(f)
f.close()

In [11]:
import warnings

arduino_port = "COM3"
#ser.close()

warnings.filterwarnings('ignore')

# Open the serial connection to the Arduino
ser = serial.Serial(arduino_port, baudrate=9600, timeout=1)

def control_guitar(case):
    if case == 'HangLoose':
        pyautoguji.hotkey('a', 'down') 
    elif case == 'Pinch':
        pyautogui.hotkey('s', 'down')
    elif case == 'Punch':
        pyautogui.hotkey('j', 'down')
    elif case == 'ThumbsUp':
        pyautogui.hotkey('k', 'down')
    elif case == 'Victory':
        pyautogui.hotkey('j', 'down')
        
# Read data from the serial port
while True:
    xiao_data = []
    for i in range(100):
        value = ser.readline().decode("utf-8").rstrip()
        if value:
            xiao_data.append(float(value))
        time.sleep(0.008)
    feature_data = feature_extraction(np.array(xiao_data))
    prediction = classifier.predict(np.array(feature_data).reshape(1, -1))
    prediction = Encoder.inverse_transform(prediction)
    print(prediction)
    #control_guitar(prediction)
    
ser.close()

SerialException: could not open port 'COM3': FileNotFoundError(2, 'O sistema não pode encontrar o arquivo especificado.', None, 2)

In [43]:
ser.close()